PASO 1 - 
CONEXION A LA API

In [85]:
import requests
import pandas as pd
#import json

#se declaran las variables para ciudad, pais, idioma y unidades para pasarlas como parametro a la api
#como idea a futuro, podria armar desde aca que itere una lista de ciudades por ejemplo para hacer crecer la db
#podria servir para responder comparacion entre ciudades en lugar de unicamente a nivel historico de una ciudad
city_name = 'Buenos Aires'
country_code = 'AR'
lang = 'es'
units = 'metric'
with open('api_key.txt','r') as p:
    api_key = p.read()

response = requests.get(
    f'https://api.openweathermap.org/data/2.5/weather?q={city_name},{country_code}&appid={api_key}&lang={lang}&units={units}')
raw_weather_data = response.json()
raw_weather_data

{'coord': {'lon': -58.3772, 'lat': -34.6132},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'cielo claro',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 21.87,
  'feels_like': 21.97,
  'temp_min': 21.07,
  'temp_max': 22.1,
  'pressure': 1013,
  'humidity': 71},
 'visibility': 10000,
 'wind': {'speed': 7.72, 'deg': 100},
 'clouds': {'all': 0},
 'dt': 1699310058,
 'sys': {'type': 1,
  'id': 8224,
  'country': 'AR',
  'sunrise': 1699260443,
  'sunset': 1699309612},
 'timezone': -10800,
 'id': 3435910,
 'name': 'Buenos Aires',
 'cod': 200}

PASO 1.1 - FUNCION DE TRANSFORMACION

In [86]:
from datetime import timedelta

def transform_data(dataframe):
    #normalizacion y concatenacion del json
    df = pd.json_normalize(dataframe)
    norm_weather_df = pd.json_normalize(df['weather'][0])
    all_weather_df = pd.concat([df, norm_weather_df], axis=1)

    #eliminacion de columnas sin uso
    reduced_weather_df = all_weather_df.drop(['base', 'timezone', 'id', 'cod', 'sys.type', 'sys.id', 'id', 'icon'], axis=1)
    
    #orden de columnas
    sorted_weather_df = reduced_weather_df[['dt','name','sys.country','coord.lon','coord.lat','main','description','main.temp','main.feels_like','main.temp_min','main.temp_max','main.pressure','main.humidity','wind.speed','wind.deg','clouds.all','sys.sunrise','sys.sunset']]

    #formato datetime de UNIX a GMT 0
    sorted_weather_df['dt'] = pd.to_datetime(sorted_weather_df['dt'], unit='s')
    sorted_weather_df['sys.sunrise'] = pd.to_datetime(sorted_weather_df['sys.sunrise'], unit='s')
    sorted_weather_df['sys.sunset'] = pd.to_datetime(sorted_weather_df['sys.sunset'], unit='s')

    #GMT -3 usando timedelta
    sorted_weather_df['dt'] = sorted_weather_df['dt'] - timedelta(hours=3)
    sorted_weather_df['sys.sunrise'] = sorted_weather_df['sys.sunrise'] - timedelta(hours=3)
    sorted_weather_df['sys.sunset'] = sorted_weather_df['sys.sunset'] - timedelta(hours=3)

    #renombrar columnas para exportar luego a Redshift
    sorted_weather_df.rename(columns = {'sys.country':'country','coord.lon':'longitude','coord.lat':'latitude','main.temp':'temp','main.feels_like':'feels_like','main.temp_min':'temp_min','main.temp_max':'temp_max','main.pressure':'pressure','main.humidity':'humidity','wind.speed':'wind_speed','wind.deg':'wind_deg','clouds.all':'clouds','sys.sunrise':'sunrise_time','sys.sunset':'sunset_time'}, inplace = True)

    return(sorted_weather_df)

2.2 - EJECUCIÓN

In [87]:
transform_data(raw_weather_data)

C:\Users\Julito\AppData\Local\Temp\ipykernel_10236\570888800.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_weather_df['dt'] = pd.to_datetime(sorted_weather_df['dt'], unit='s')
C:\Users\Julito\AppData\Local\Temp\ipykernel_10236\570888800.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_weather_df['sys.sunrise'] = pd.to_datetime(sorted_weather_df['sys.sunrise'], unit='s')
C:\Users\Julito\AppData\Local\Temp\ipykernel_10236\570888800.py:18: SettingWithCopyWarning: 
A value is trying to 

,dt,name,country,longitude,latitude,main,description,temp,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds,sunrise_time,sunset_time
0,2023-11-06 19:34:18,Buenos Aires,AR,-58.3772,-34.6132,Clear,cielo claro,21.87,21.97,21.07,22.1,1013,71,7.72,100,0,2023-11-06 05:47:23,2023-11-06 19:26:52


2.3 - VARIABLE FUERA DE LA FUNCION PARA LUEGO CARGAR EN REDSHIFT

In [88]:
final_weather_df = transform_data(raw_weather_data)

C:\Users\Julito\AppData\Local\Temp\ipykernel_10236\570888800.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_weather_df['dt'] = pd.to_datetime(sorted_weather_df['dt'], unit='s')
C:\Users\Julito\AppData\Local\Temp\ipykernel_10236\570888800.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_weather_df['sys.sunrise'] = pd.to_datetime(sorted_weather_df['sys.sunrise'], unit='s')
C:\Users\Julito\AppData\Local\Temp\ipykernel_10236\570888800.py:18: SettingWithCopyWarning: 
A value is trying to 

PASO 2 - CONEXIÓN A REDSHIFT

In [89]:
import psycopg2

url='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
data_base='data-engineer-database'
user='juliocfurtado93_coderhouse'
with open('pwd.txt','r') as file:
    pwd= file.read()

try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print('Conexión a Redshift exitosa!')
    
except Exception as e:
    print('No se pudo conectar a Redshift.')
    print(e)

Conexión a Redshift exitosa!


2.1 - FUNCION DE CARGA

In [ ]:
#me falta que pueda manejar duplicados usando una pk
#de encontrar un duplicado, deberia reemplazarlo con lo ultimo que se intente cargar

In [90]:
from psycopg2.extras import execute_values
from psycopg2 import sql

def redshift_carga(conn, table_name, dataframe):
    #almacena los tipos de datos del df
    dtypes= dataframe.dtypes

    #las columnas del df
    columnas= list(dtypes.index )

    #los valores dentro de los tipos de dato
    tipos_pre= list(dtypes.values)

    #mapa de tipos para redshift
    mapa = {
        'int64': 'INT',
        'float64': 'FLOAT',
        'object': 'VARCHAR(50)',
        'datetime64[ns]': 'TIMESTAMP'
        }
    
    #itera para asignarle un valor del mapa a cada dtype de esa variable
    sql_tipos = [mapa[str(dtype)] for dtype in tipos_pre]

    #arma los nombres de las columnas en sql para poder armar la sentencia de creacion
    columnas_redshift = [f"{name} {data_type}" for name, data_type in zip(columnas, sql_tipos)]

    #schema por si no existiese la tabla
    table_schema = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join(columnas_redshift)}
        );
        """
    
    #crea la tabla si no existe
    cur = conn.cursor()
    cur.execute(table_schema)

    #guarda los valores en tuplas para insertar
    values = [tuple(x) for x in dataframe.to_numpy()]

    #define el insert usando sql de psycopg2 (itera con las columnas tambien)
    insert_sql = sql.SQL("INSERT INTO {} ({}) VALUES %s").format(
        sql.Identifier(table_name),
        sql.SQL(', ').join(map(sql.Identifier, columnas))
    )

    #execute values ejecuta la inserción en la tabla
    cur.execute("BEGIN")
    execute_values(cur, insert_sql, values)
    cur.execute("COMMIT")
    print('Carga completada en Redshift!')

2.2 - EJECUCIÓN

In [91]:
redshift_carga(conn=conn, table_name='open_weather_v2', dataframe=final_weather_df)

Carga completada en Redshift!


2.3 - CIERRE DE CONEXIÓN

In [92]:
conn.close()